In [9]:
#import packages
import os
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

ini_data = pd.ExcelFile('Построение рейтинговой модели.xlsx')
df = ini_data.parse('data2')

In [10]:
def ks(data=None,target=None, prob=None):
    data['target0'] = 1 - data[target]
    data['bucket'] = pd.qcut(data[prob], 10)
    grouped = data.groupby('bucket', as_index = False)
    kstable = pd.DataFrame()
    kstable['min_prob'] = grouped.min()[prob]
    kstable['max_prob'] = grouped.max()[prob]
    kstable['events']   = grouped.sum()[target]
    kstable['nonevents'] = grouped.sum()['target0']
    kstable = kstable.sort_values(by="min_prob", ascending=False).reset_index(drop = True)
    kstable['event_rate'] = (kstable.events / data[target].sum()).apply('{0:.2%}'.format)
    kstable['nonevent_rate'] = (kstable.nonevents / data['target0'].sum()).apply('{0:.2%}'.format)
    kstable['cum_eventrate']=(kstable.events / data[target].sum()).cumsum()
    kstable['cum_noneventrate']=(kstable.nonevents / data['target0'].sum()).cumsum()
    kstable['KS'] = np.round(kstable['cum_eventrate']-kstable['cum_noneventrate'], 3) * 100

    #Formating
    kstable['cum_eventrate']= kstable['cum_eventrate'].apply('{0:.2%}'.format)
    kstable['cum_noneventrate']= kstable['cum_noneventrate'].apply('{0:.2%}'.format)
    kstable.index = range(1,11)
    kstable.index.rename('Decile', inplace=True)
    pd.set_option('display.max_columns', 9)
    print(kstable)
    
    #Display KS
    from colorama import Fore
    print(Fore.RED + "KS is " + str(max(kstable['KS']))+"%"+ " at decile " + str((kstable.index[kstable['KS']==max(kstable['KS'])][0])))
    return(kstable)

In [11]:
mydf = ks(data=df,target="y", prob="p")

        min_prob  max_prob  events  nonevents event_rate nonevent_rate  \
Decile                                                                   
1       0.860242  0.985133      29          3     10.36%         0.50%   
2       0.752615  0.860158     135         10     48.21%         1.65%   
3       0.521948  0.751636      40         49     14.29%         8.10%   
4       0.276042  0.521800      30         58     10.71%         9.59%   
5       0.159324  0.275990      22         66      7.86%        10.91%   
6       0.092190  0.155566      12         77      4.29%        12.73%   
7       0.059672  0.091865       5         83      1.79%        13.72%   
8       0.030212  0.059331       3         86      1.07%        14.21%   
9       0.011676  0.029394       3         85      1.07%        14.05%   
10      0.000274  0.011538       1         88      0.36%        14.55%   

       cum_eventrate cum_noneventrate    KS  
Decile                                       
1             10.36

In [12]:
from scipy.stats import ks_2samp

ks_2samp(df.loc[df.y==0,"p"], df.loc[df.y==1,"p"])

Ks_2sampResult(statistic=0.649793388429752, pvalue=6.265008721931893e-72)

We can reject the null hypothesis and concluding distribution of events and non-events is different.